# ARTS2

The backup recovery device. It has a single accelerometer and a pressure sensor.

## Acceleration

We can look at it's accelerometer output compared to our primary ADIS IMU.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, argmax, add, average
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665
FEET2METERS = 0.3048

columns = loadtxt("../ARTS2/2015-07-19_ARTS2_Flight_Data_Interpreted_Data.csv",
                  delimiter=',', comments='#',unpack=True)
arts_time = columns[0]
arts_accel = columns[1]
arts_vel = columns[2]
arts_ialt = columns[3]
arts_palt = multiply(columns[4], FEET2METERS)


# ADIS data for reference
columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
acc_x, acc_y, acc_z = columns[6:9]

t_0 = 117853569585227

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)

# lightly lowpass
from scipy.signal import butter, lfilter, freqz

# Filter requirements.
order = 6
fs = 819.2       # sample rate, Hz
cutoff = 20   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq

# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_x_filter = lfilter(b, a, acc_x)

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"ARTS2 Accelerometer")
plt.ylabel(r"Acceleration [m/s${}^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(timestamp[200:-200], subtract(acc_x_filter[200:-200], g_0), 'k-', alpha=0.3, lw=2.0, label="ADIS IMU Vertical Acceleration")
plt.plot(arts_time, multiply(arts_accel, g_0), alpha=0.75, label="ARTS2 Vertical Acceleration")

plt.xlim([-5,42])
plt.ylim([-50,130])
ax1.legend(loc=1)
plt.show()

## Altitude

The ARTS2 uses it's pressure sensor to approximate altitude using it's internal atmosphere model and calibration.

In [ ]:
print """The maximum altitude recorded by the pressure sensor on the ARTS2 was %0.0f meters.
""" % (max(arts_palt))

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"ARTS2 Pressure Altitude")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

#plt.plot(timestamp[200:-200], subtract(acc_x_filter[200:-200], g_0), 'k-', alpha=0.3, lw=2.0, label="ADIS IMU Vertical Accel")
plt.plot(arts_time, divide(arts_palt, 1000.0), alpha=0.75, label="ARTS2 Vertical Acceleration")

plt.xlim([-5,42])
#plt.ylim([-50,130])
#ax1.legend(loc=2)
plt.show()